# Utilizing tools in LangChain

In [ ]:
from langchain.agents import tool, Tool, create_react_agent, create_structured_chat_agent, AgentExecutor
from langchain_openai import OpenAI
from langchain import hub
from langchain.tools import StructuredTool

import os

openai_api_key = os.getenv('OPENAI_API_KEY') 

## Creating custom tools

In [ ]:
# Define the calculate_ltv tool function
@tool
def calculate_ltv(company_name: str) -> str:
    """Generate the LTV for a company."""
    avg_churn = 0.25
    avg_revenue = 1000
    historical_LTV = avg_revenue / avg_churn

    report = f"LTV Report for {company_name}\n"
    report += f"Avg. churn: ${avg_churn}\n"
    report += f"Avg. revenue: ${avg_revenue}\n"
    report += f"historical_LTV: ${historical_LTV}\n"
    return report

# Define the tools list
tools = [Tool(name="LTVReport",
              func=calculate_ltv,
              description="Use this for calculating historical LTV.")]

# Initialize the appropriate agent type
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0, openai_api_key=openai_api_key)

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm,
    tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True, max_iterations=5
)

input = (
    "Run a financial report that calculates historical LTV for Hooli"
)
agent_executor.invoke({"input": input})

## Scaling custom tools

In [ ]:
def calculate_wellness_score(sleep_hours, exercise_minutes, healthy_meals, stress_level):
    """Calculate a Wellness Score based on sleep, exercise, nutrition, and stress management."""
    max_score_per_category = 25

    sleep_score = min(sleep_hours / 8 * max_score_per_category, max_score_per_category)
    exercise_score = min(exercise_minutes / 30 * max_score_per_category, max_score_per_category)
    nutrition_score = min(healthy_meals / 3 * max_score_per_category, max_score_per_category)
    stress_score = max_score_per_category - min(stress_level / 10 * max_score_per_category, max_score_per_category)

    total_score = sleep_score + exercise_score + nutrition_score + stress_score
    return total_score

# Create a structured tool from calculate_wellness_score()
tools = [StructuredTool.from_function(calculate_wellness_score)]

# Initialize the appropriate agent type and tool set
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0, openai_api_key=openai_api_key)

prompt = hub.pull("hwchase17/react")

agent = create_structured_chat_agent(
    llm,
    tools,
    prompt=prompt,
)

wellness_tool = tools[0]
result = wellness_tool.func(sleep_hours=8, exercise_minutes=14, healthy_meals=10, stress_level=20)
print(result)

## Formatting tools as OpenAI functions

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

# Create an LTVDescription class to manually add a function description
class LTVDescription(BaseModel):
    query: str = Field(description='Calculate an extremely simple historical LTV')

# Format the calculate_ltv tool function so it can be used by OpenAI models
@tool(args_schema=LTVDescription)
def calculate_ltv(company_name: str) -> str:
    """Generate the LTV for a company to pontificate with."""
    avg_churn = 0.25
    avg_revenue = 1000
    historical_LTV = avg_revenue / avg_churn

    report = f"Pontification Report for {company_name}\n"
    report += f"Avg. churn: ${avg_churn}\n"
    report += f"Avg. revenue: ${avg_revenue}\n"
    report += f"historical_LTV: ${historical_LTV}\n"
    return report

print(convert_to_openai_function(calculate_ltv))